[View in Colaboratory](https://colab.research.google.com/github/SwapnilSParkhe/Project-Image_Caption_Generation/blob/master/Building&Fitting_Model.ipynb)

# Building Model Architecture and Fitting to Data

**Checking GPU status**

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3396063252524753574, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11288962663
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1124319443613624125
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

## Analytical Data Creation

**Image ID or Identifiers**

In [5]:
#Uploading relevant files from local to cloud (using google.colab lib)

#Library for Uploading data from local to cloud
from google.colab import files

#Upload train image text
files.upload()   #upload files 

#Upload valid image text
files.upload()   #upload files

Saving Flickr_8k.trainImages.txt to Flickr_8k.trainImages.txt


Saving Flickr_8k.devImages.txt to Flickr_8k.devImages.txt


{'Flickr_8k.devImages.txt': b'2090545563_a4e66ec76b.jpg\n3393035454_2d2370ffd4.jpg\n3695064885_a6922f06b2.jpg\n1679557684_50a206e4a9.jpg\n3582685410_05315a15b8.jpg\n1579798212_d30844b4c5.jpg\n269650644_059a84ece5.jpg\n2596619849_7b635dd310.jpg\n2635164923_2a774f7854.jpg\n3583321426_f373c52161.jpg\n2735792721_b8fe85e803.jpg\n3484841598_e26ee96aab.jpg\n2490687446_9d46fdf5a9.jpg\n2987328689_96a2d814f1.jpg\n2992614450_b5a6692239.jpg\n2391812384_7429b5e567.jpg\n3665179773_dd217416fc.jpg\n3515904775_f8acc5909e.jpg\n3616808182_fb4eaec778.jpg\n3652150541_8fb5a3a5d1.jpg\n3693961165_9d6c333d5b.jpg\n2895700779_fac1d9d278.jpg\n2474092890_6c0781a8ed.jpg\n3420469425_2980b4cd30.jpg\n537222436_ab406234ce.jpg\n3419634480_c390f62a6e.jpg\n2659183350_730951f740.jpg\n1178705300_c224d9a4f1.jpg\n989851184_9ef368e520.jpg\n535123126_c06c1ab9bf.jpg\n3226541300_6c81711e8e.jpg\n3457784061_8f77f43a9c.jpg\n3364715316_df8c20bac1.jpg\n233361142_d9d5f0cae9.jpg\n634891010_9fa189effb.jpg\n2977379863_2e8d7a104e.jpg\n3663

In [0]:
#Importing file: reading the content into Py file
def import_file(input_file):
    file=open(input_file,'r')   #creating a bridge btwn OS and Py files
    content=file.read()   #reading content via the bridge
    file.close()   #closing the bridge
    return content

imported_train=import_file('Flickr_8k.trainImages.txt') 
imported_valid=import_file('Flickr_8k.devImages.txt')    

#Creating a set of image-IDs
def create_img_set(file):
    imgID_set=list()
    for item in file.split('\n'):   #accessing line by line
        if len(item)<1:   #rejecting empty spaces
            continue
        imgID=item.split('.')[0]   #only taking imgID (rejecting 'jpg')
        imgID_set.append(imgID)   #appending imgIDs to imgID_set
    return set(imgID_set)

imgID_trainset=create_img_set(imported_train)
imgID_validset=create_img_set(imported_valid)

**Importing previously created files (from PreprocessingData NoteBook): Img desc and Img features**

In [7]:
#Uploading relevant files from local to cloud (using google.colab lib)

#Library for Uploading data from local to cloud
from google.colab import files

#Upload cleaned organised text file (from Text precprosssing step)
files.upload()   #upload files 

#Upload features or weights file (from Image preprocessing step)
files.upload()   #upload files

Saving cln_orgnse_text.txt to cln_orgnse_text.txt


Saving features.pkl to features.pkl


In [0]:
#Importing image desc files for this image data germane to training set
def import_prepro_desc(prepro_file, dataset):
    file=import_file(prepro_file)
    desc=dict()
    for item in file.split('\n'):
        tokens=item.split()   #splitting by whitespaces
        image_ID,image_desc=tokens[0],tokens[1:]   #separating ID, desc
        if image_ID in dataset:   #inner join imgID & training imgID 
            if image_ID not in desc:   #new list for new image_ID key 
                desc[image_ID]=list()
            desc_='start ' + ' '.join(image_desc)+' end'   #wrap in tokens
            desc[image_ID].append(desc_)
    return desc

desc_train=import_prepro_desc('cln_orgnse_text.txt',imgID_trainset)
desc_valid=import_prepro_desc('cln_orgnse_text.txt',imgID_validset)

#Importing image features for this image data germane to training set
from pickle import load
def import_features(feature_file, dataset):
    all_features = load(open(feature_file, 'rb'))  #load all features
    features = {k: all_features[k] for k in dataset} #inner join
    return features

feature_train=import_features('features.pkl',imgID_trainset) #used later
feature_valid=import_features('features.pkl',imgID_validset) #used later

**Training data manipulations: Creating a custom Tokeizer function: Tokenizing descriptions**

In [7]:
#Creating a simple list of desc from dict of desc
def dict2list(input_dict):
    desc_list=list()
    for key in input_dict.keys():
        [desc_list.append(d) for d in input_dict[key]]
    return desc_list

desc_train_list=dict2list(desc_train)

#tokeinizing (could be improved by filetring english stopwords later)
#Note: turning each text into sequence of integers (integer: token ID)
from keras.preprocessing.text import Tokenizer
def tokenize(input_list):
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(input_list)
    return tokenizer

tokenizer=tokenize(desc_train_list) #to be used later
vocab_size=len(tokenizer.word_index)+1 #to be used later
print("Vocab Size:",vocab_size)

#Length of the description with the most words
def max_length(desc_list):
    max_len=max([len(item.split()) for item in desc_list])
    return max_len
max_length = max_length(desc_train_list) #to be used later
print('Description Length', max_length)

#Longest desc check
def longest_desc(desc_list):
    max_length=max([len(item.split()) for item in desc_list])
    print("Max_len:",max_length)
    print("Desc:", [item for item in desc_list if len(item.split())==max_length])

longest_desc(desc_train_list)

Vocab Size: 7264
Description Length 33
Max_len: 33
Desc: ['start an man wearing green sweatshirt and blue vest is holding up dollar bills in front of his face while standing on busy sidewalk in front of group of men playing instruments end']


**LSTM's Analytical Dataset: Input(ImageID and Seq_item)-Ouput(SeqWord) data**

In [0]:
#Creating ADS for LSTM: Input(Image_ID and Seq_item)-Ouput(SeqWord)
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
def create_ADS(tokenizer, max_length, desc_dict, img):
    X_img_ID, X_desc_item, y=list(), list(), list()
    for key, desc_list in desc_dict.items(): #key to access desc, img
        for desc in desc_list:
            seq=tokenizer.texts_to_sequences([desc])[0] #encoding seq
            for i in range(1,len(seq)):#split seq into multi X,y pairs
                in_seq, out_seq=seq[:i], seq[i] #desc input-output pair
                in_seq=pad_sequences([in_seq], maxlen=max_length)[0]
                out_seq=to_categorical([out_seq], num_classes=vocab_size)[0]
                X_img_ID.append(img[key][0]) #same imgID for multi X-y pair
                X_desc_item.append(in_seq)  #multi X-y pairs encoding
                y.append(out_seq)   #oneHot encoded version of output word
    return np.array(X_img_ID), np.array(X_desc_item), np.array(y)

#X_img_ID_train, X_desc_item_train, y_train = create_ADS(tokenizer, max_length, desc_train, feature_train)
X_img_ID_valid, X_desc_item_valid, y_valid = create_ADS(tokenizer, max_length, desc_valid, feature_valid)

## Defining the model (Merge Model of Embeddings+LSTMs with CNN penultimate layer)
**Note:** Combines both the encoded form (features) of the image input with the encoded form (context) of the text description generated so far; Combination of these two encoded inputs is then used by a very simple decoder model to generate the next word in the sequence

![Merge Model of Image Captioning](https://i.pinimg.com/originals/35/8b/dc/358bdc11e71f8c78632560c7c819919d.png)

**Importing relevant libraries**

In [0]:
from keras.layers import Input, Dropout, Dense #feat. encoding
from keras.layers import Embedding, Dropout, LSTM #desc. encoding
from keras.layers.merge import add #decoding
from keras.models import Model #Model-Input-Output architecture

In [10]:
def build_model_arch(vocab_size, max_length):
    #Encoder Models (Img-Feat and Desc Encoding)
    #1.Image feature extractor model
    feat_input=Input(shape=(4096,))
    feat_1=Dropout(0.5)(feat_input)
    feat_2=Dense(256, activation='relu')(feat_1)

    #2.Embedding+LSTM sequence model
    desc_input=Input(shape=(max_length,))
    desc_1=Embedding(vocab_size, 256, mask_zero=True)(desc_input)
    desc_2=Dropout(0.5)(desc_1)
    desc_3=LSTM(256)(desc_2)

    #Decoder Model ('adding' above encoding model layers; with FFNs)
    deco_1=add([feat_2, desc_3]) #adding element wise for both vectors
    deco_2=Dense(256, activation='relu')(deco_1)
    output=Dense(vocab_size, activation='softmax')(deco_2)

    #Creating Model-Input-Output architecture; Compiling (with loss, opt.)
    model=Model(inputs=[feat_input, desc_input], outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    #Summarizing and Plotting model
    print(model.summary())
    return model

model = build_model_arch(vocab_size, max_length)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 33)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 33, 256)      1859584     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4096)         0           input_1[0][0]                    
__________________________________________________________________________________________________
dropout_2 

## Fitting Model

In [11]:
#Defining checkpoint callback
from keras.callbacks import ModelCheckpoint
filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

#Fitting model to data (along side validation loss checks)
model.fit([X_img_ID_train, X_desc_item_train], y_train, epochs=5, verbose=1, callbacks=[checkpoint], 
          validation_data=([X_img_ID_valid, X_desc_item_valid], y_valid))

Train on 50835 samples, validate on 50835 samples
Epoch 1/5
24384/50835 [=============>................] - ETA: 1:50 - loss: 5.5770

50835/50835 [==============================] - 267s 5ms/step - loss: 5.2323 - val_loss: 4.4434

Epoch 00001: val_loss improved from inf to 4.44338, saving model to model-ep001-loss5.232-val_loss4.443.h5
Epoch 2/5
 4672/50835 [=>............................] - ETA: 3:12 - loss: 4.4181

50835/50835 [==============================] - 267s 5ms/step - loss: 4.2924 - val_loss: 3.9040



Epoch 00002: val_loss improved from 4.44338 to 3.90402, saving model to model-ep002-loss4.292-val_loss3.904.h5
Epoch 3/5
16544/50835 [========>.....................] - ETA: 2:22 - loss: 3.9252

50835/50835 [==============================] - 266s 5ms/step - loss: 3.9023 - val_loss: 3.5896

Epoch 00003: val_loss improved from 3.90402 to 3.58964, saving model to model-ep003-loss3.902-val_loss3.590.h5
Epoch 4/5
 2656/50835 [>.............................] - ETA: 3:18 - loss: 3.6325

50835/50835 [==============================] - 266s 5ms/step - loss: 3.6434 - val_loss: 3.3596



Epoch 00004: val_loss improved from 3.58964 to 3.35963, saving model to model-ep004-loss3.643-val_loss3.360.h5
Epoch 5/5
16544/50835 [========>.....................] - ETA: 2:22 - loss: 3.4195

50835/50835 [==============================] - 269s 5ms/step - loss: 3.4394 - val_loss: 3.1396

Epoch 00005: val_loss improved from 3.35963 to 3.13962, saving model to model-ep005-loss3.439-val_loss3.140.h5
